<a href="https://colab.research.google.com/github/pearpare/sherlock-lstm/blob/main/lstm_project_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elisabeth Kam (etk45) 

I decided to try the LSTM model using the author's suggested window size of 100 characters. I also tried a model with 3 layers and window size of 100. Finally, I tried a smaller window size of 50 on both the 2 layer and 3 layer model. Based on the text generated, text clarity and accuracy appeared best with the 3 layer model. The difference in window size did not seem to alter the text a lot. I did note, though, that the model with the smaller window size trained much faster. Also, for some reason the window size = 50 and 2 layer model experiment needed to try generate a text prompt 2 or 3 times before legible text was generated with the prompt. 

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.utils.data as data 

In [2]:
torch.cuda.is_available()

True

Experiment with window size = 100 starts here. 

In [114]:
filename = "/sherlock.txt"
sh_raw_txt = open(filename, 'r', encoding = 'utf-8').read()
sh_raw_txt = sh_raw_txt.lower()
sh_raw_txt = sh_raw_txt[:50000]
chars = sorted(list(set(sh_raw_txt)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [115]:
n_chars = len(sh_raw_txt)
n_vocab = len(chars)
print("Total characters: ", n_chars)
print("Total vocab: ", n_vocab)

Total characters:  50000
Total vocab:  44


In [116]:
#prepare the dataset of input to output pairs encoded as integers
char_seq_len = 100 #larger window size 
X_data = []
y_data = []

for i in range(0, n_chars - char_seq_len, 1):
    seq_in = sh_raw_txt[i:i + char_seq_len]
    seq_out = sh_raw_txt[i + char_seq_len]
    X_data.append([char_to_int[char] for char in seq_in])
    y_data.append(char_to_int[seq_out])
    
n_patterns = len(X_data)
print("Total patterns: ", n_patterns)

Total patterns:  49900


Model with 2 layers is here. 

In [126]:
class bookModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout = 0.2)
        self.dropout = nn.Dropout(0.2) #could try changing droput values for fun 
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x): 
        x, _ = self.lstm(x)
        # takes only the last output 
        x = x[:, -1, :]
        # produce output 
        x = self.linear(self.dropout(x))
        return x 

In [117]:
X = torch.tensor(X_data, dtype=torch.float32).reshape(n_patterns, char_seq_len, 1)
X = X / float(n_vocab)
y = torch.tensor(y_data)
print(X.shape, y.shape)

torch.Size([49900, 100, 1]) torch.Size([49900])


In [80]:
n_epochs = 50
batch_size = 128 
model = bookModel()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(device)
model.to(device)

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle = True, batch_size=batch_size)

best_model = None
best_loss = np.inf

for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader: 
        y_pred = model(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #Validation Time
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.3f" % (epoch, loss))
torch.save([best_model, char_to_int], "single-char2.pth")

Epoch 0: Cross-entropy: 146781.094
Epoch 1: Cross-entropy: 135529.297
Epoch 2: Cross-entropy: 130175.898
Epoch 3: Cross-entropy: 126038.141
Epoch 4: Cross-entropy: 122130.812
Epoch 5: Cross-entropy: 119098.273
Epoch 6: Cross-entropy: 116268.281
Epoch 7: Cross-entropy: 112764.578
Epoch 8: Cross-entropy: 109724.141
Epoch 9: Cross-entropy: 107512.188
Epoch 10: Cross-entropy: 104617.172
Epoch 11: Cross-entropy: 102173.609
Epoch 12: Cross-entropy: 100138.430
Epoch 13: Cross-entropy: 99255.789
Epoch 14: Cross-entropy: 95459.695
Epoch 15: Cross-entropy: 93677.250
Epoch 16: Cross-entropy: 91449.516
Epoch 17: Cross-entropy: 89326.305
Epoch 18: Cross-entropy: 87645.375
Epoch 19: Cross-entropy: 86385.469
Epoch 20: Cross-entropy: 84793.602
Epoch 21: Cross-entropy: 82231.797
Epoch 22: Cross-entropy: 80089.805
Epoch 23: Cross-entropy: 77776.555
Epoch 24: Cross-entropy: 76789.234
Epoch 25: Cross-entropy: 74719.570
Epoch 26: Cross-entropy: 73619.094
Epoch 27: Cross-entropy: 70780.656
Epoch 28: Cross-e

In [84]:
best_model, char_to_int, torch.load("single-char2.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)

<All keys matched successfully>

In [85]:
#generate a prompt here 
file = "/sherlock.txt"
raw_txt2 = open(file, 'r', encoding = 'utf-8').read()
raw_txt2 = raw_txt2.lower()
raw_txt2 = raw_txt2[:50000]
seq_len = 100
start = np.random.randint(0, len(raw_txt2)-seq_len)
prompt = raw_txt2[start:start+seq_len]
pattern = [char_to_int[c] for c in prompt]

In [86]:
model.eval()
print("Prompt:")
print(prompt)
print("Prompt ends here.")
print("\n")
print("Result:")
with torch.no_grad():
  for i in range(1000):
    #format input array of int into pytorch tensor 
    x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
    x = torch.tensor(x, dtype=torch.float32)
    #genreate logits as output from the model 
    pred = model(x.to(device))
    #convert logits into one character
    index = int(pred.argmax())
    result = int_to_char[index]
    print(result, end="")
    #append the new character into the prompt for the next iteration
    pattern.append(index)
    pattern = pattern[1:]

print()
print("Done.")

Prompt:
st go to life itself, which is always far more
daring than any effort of the imagination."

"a propo
Prompt ends here.


Result:
witeon which i should nere that i have aeen oo doubt that the was there in a dener alear in a ger casi. and i have not seens her heai. 
"then, what dod howerest to thmek in in to make a mystery." 
"i am suree that so the coon which i have aeen watch form the street. 
"as the cound to bone in the mant sery wour hes sodeess to and down in the mady, but she sase the door of the sor dound have been carsed me aoong the mat who werk foom the house of aviony ootilngs, and i was a fouble so ae on the sodee of av the house. and the cooner of the street. 

"then ther?"

"i was allaidd toundrs." 
"i was allara"

"to an and that in the len's puine mer. and the coack of his singular. 
"as ie spok there is a lans so be an active me and lnto besirting then the photograph and amleared so the shouger. "i said the lang of the sor dound have been harde to she hnuse of a

I think the text generated is okay. I'm able to identify some sentences and words even when misspelled. 

Model with 3 layers is here. 

In [138]:
class book2Model(nn.Module): #created model with 3 layers 
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=3, batch_first=True, dropout = 0.2)
        self.dropout = nn.Dropout(0.2) #could try changing droput values for fun 
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x): 
        x, _ = self.lstm(x)
        # takes only the last output 
        x = x[:, -1, :]
        # produce output 
        x = self.linear(self.dropout(x))
        return x 

Experiment with 3 layer model. 

In [119]:
n_epochs = 50
batch_size = 128 
model = book2Model()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(device)
model.to(device)

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle = True, batch_size=batch_size)

best_model = None
best_loss = np.inf

for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader: 
        y_pred = model(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #Validation Time
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.3f" % (epoch, loss))
torch.save([best_model, char_to_int], "single-char-3.pth")

Epoch 0: Cross-entropy: 150704.516
Epoch 1: Cross-entropy: 132908.656
Epoch 2: Cross-entropy: 125894.719
Epoch 3: Cross-entropy: 120314.141
Epoch 4: Cross-entropy: 115529.609
Epoch 5: Cross-entropy: 109486.078
Epoch 6: Cross-entropy: 105817.461
Epoch 7: Cross-entropy: 104835.742
Epoch 8: Cross-entropy: 97887.938
Epoch 9: Cross-entropy: 94645.336
Epoch 10: Cross-entropy: 91818.727
Epoch 11: Cross-entropy: 88327.641
Epoch 12: Cross-entropy: 85569.727
Epoch 13: Cross-entropy: 82476.875
Epoch 14: Cross-entropy: 79475.359
Epoch 15: Cross-entropy: 77831.297
Epoch 16: Cross-entropy: 74107.625
Epoch 17: Cross-entropy: 72303.164
Epoch 18: Cross-entropy: 69078.734
Epoch 19: Cross-entropy: 67962.422
Epoch 20: Cross-entropy: 64573.617
Epoch 21: Cross-entropy: 61997.953
Epoch 22: Cross-entropy: 60507.984
Epoch 23: Cross-entropy: 57628.055
Epoch 24: Cross-entropy: 55925.523
Epoch 25: Cross-entropy: 53027.613
Epoch 26: Cross-entropy: 51084.098
Epoch 27: Cross-entropy: 49110.070
Epoch 28: Cross-entrop

In [123]:
best_model, char_to_int, torch.load("single-char-3.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)

<All keys matched successfully>

In [124]:
#generate a prompt here 
file3 = "/sherlock.txt"
raw_txt3 = open(file3, 'r', encoding = 'utf-8').read()
raw_txt3 = raw_txt3.lower()
raw_txt3 = raw_txt3[:50000]
seq_len = 100
start = np.random.randint(0, len(raw_txt3)-seq_len)
prompt = raw_txt3[start:start+seq_len]
pattern = [char_to_int[c] for c in prompt]

In [125]:
model.eval()
print("Prompt:")
print(prompt)
print("Prompt ends here.")
print("\n")
print("Result:")
with torch.no_grad():
  for i in range(1000):
    #format input array of int into pytorch tensor 
    x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
    x = torch.tensor(x, dtype=torch.float32)
    #genreate logits as output from the model 
    pred = model(x.to(device))
    #convert logits into one character
    index = int(pred.argmax())
    result = int_to_char[index]
    print(result, end="")
    #append the new character into the prompt for the next iteration
    pattern.append(index)
    pattern = pattern[1:]

print()
print("Done.")

Prompt:

aisle like any other idler who has dropped into a church.
suddenly, to my surprise, the three at th
Prompt ends here.


Result:
e tittant fallly and sushed to the chamber which
had been ly rllplight in the character of an amiable and laughed to the cegebrated. the coiver look of his own high-power
with a fear half any other purpise and colpanion, but the steps of miss at the signal i tosk the soom and alose so she ceattifam which
had been lys upon the steps; she watched us with the street. and i have not seen her since. i rose, and i mean
to see holmes as he lay
upon the smaller crimes, and in a gentleman wpon his chair and paserved in men. and the man who wrote the soom and cound seven fuodred of street, and the lady of the steps of my mwst
be on the sight side, well furnished, with a bhepce puttosseo, h ment the staged oe steeertande
ar the coor of briony lodge, as it munt ae aought under half a crown a packet. it is alwo in the least interested. but
the could not lerely wha

I think for this model the performance was a little better. The sentences resemble the original text more closely (at least to my knowledge of Sherlock Holmes). 

Experiment with window size = 50 starts here. 

In [127]:
filename50 = "/sherlock.txt"
raw_txt50 = open(filename50, 'r', encoding = 'utf-8').read()
raw_txt50 = raw_txt50.lower()
raw_txt50 = raw_txt50[:50000]
chars = sorted(list(set(raw_txt50)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [128]:
n_chars = len(raw_txt50)
n_vocab = len(chars)
print("Total characters: ", n_chars)
print("Total vocab: ", n_vocab)

Total characters:  50000
Total vocab:  44


In [129]:
char_seq_len = 50
X_data = []
y_data = []

for i in range(0, n_chars - char_seq_len, 1):
    seq_in = raw_txt50[i:i + char_seq_len]
    seq_out = raw_txt50[i + char_seq_len]
    X_data.append([char_to_int[char] for char in seq_in])
    y_data.append(char_to_int[seq_out])
    
n_patterns = len(X_data)
print("Total patterns: ", n_patterns)

Total patterns:  49950


In [130]:
X = torch.tensor(X_data, dtype=torch.float32).reshape(n_patterns, char_seq_len, 1)
X = X / float(n_vocab)
y = torch.tensor(y_data)
print(X.shape, y.shape)

torch.Size([49950, 50, 1]) torch.Size([49950])


In [131]:
n_epochs = 50
batch_size = 128 
model50 = bookModel()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(device)
model50.to(device)

optimizer = optim.Adam(model50.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle = True, batch_size=batch_size)

best_model = None
best_loss = np.inf

for epoch in range(n_epochs):
    model50.train()
    for X_batch, y_batch in loader: 
        y_pred = model50(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #Validation Time
    model50.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model50(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model50.state_dict()
        print("Epoch %d: Cross-entropy: %.3f" % (epoch, loss))
torch.save([best_model, char_to_int], "single-char-50.pth")

Epoch 0: Cross-entropy: 145674.844
Epoch 1: Cross-entropy: 134426.000
Epoch 2: Cross-entropy: 129494.883
Epoch 3: Cross-entropy: 125980.109
Epoch 4: Cross-entropy: 121899.492
Epoch 5: Cross-entropy: 118436.031
Epoch 6: Cross-entropy: 115019.148
Epoch 7: Cross-entropy: 112203.992
Epoch 8: Cross-entropy: 109002.766
Epoch 9: Cross-entropy: 106233.922
Epoch 10: Cross-entropy: 104881.367
Epoch 11: Cross-entropy: 101396.672
Epoch 12: Cross-entropy: 99150.281
Epoch 13: Cross-entropy: 96492.945
Epoch 14: Cross-entropy: 94176.617
Epoch 15: Cross-entropy: 92015.469
Epoch 16: Cross-entropy: 89741.727
Epoch 17: Cross-entropy: 88383.641
Epoch 18: Cross-entropy: 85589.969
Epoch 19: Cross-entropy: 84468.227
Epoch 20: Cross-entropy: 82199.102
Epoch 21: Cross-entropy: 81272.930
Epoch 22: Cross-entropy: 78466.141
Epoch 23: Cross-entropy: 76956.281
Epoch 24: Cross-entropy: 74749.031
Epoch 25: Cross-entropy: 72820.562
Epoch 26: Cross-entropy: 70892.922
Epoch 27: Cross-entropy: 69317.289
Epoch 28: Cross-en

In [135]:
best_model, char_to_int, torch.load("single-char-50.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model50.load_state_dict(best_model)

<All keys matched successfully>

In [136]:
#generate a prompt here 
file50 = "/sherlock.txt"
raw_2txt = open(file50, 'r', encoding = 'utf-8').read()
raw_2txt = raw_2txt.lower()
raw_2txt = raw_2txt[:50000]
seq_len = 50
start = np.random.randint(0, len(raw_2txt)-seq_len)
prompt = raw_2txt[start:start+seq_len]
pattern = [char_to_int[c] for c in prompt]

In [137]:
model50.eval()
print("Prompt:")
print(prompt)
print("Prompt ends here.")
print("\n")
print("Result:")
with torch.no_grad():
  for i in range(1000):
    #format input array of int into pytorch tensor 
    x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
    x = torch.tensor(x, dtype=torch.float32)
    #genreate logits as output from the model 
    pred = model(x.to(device))
    #convert logits into one character
    index = int(pred.argmax())
    result = int_to_char[index]
    print(result, end="")
    #append the new character into the prompt for the next iteration
    pattern.append(index)
    pattern = pattern[1:]

print()
print("Done.")

Prompt:
struck savagely at each other with their fists and
Prompt ends here.


Result:
 sticks. holmes whistled.

"a pair, by the sousd it ore of the lont lerters what he was a froup of betpared with a fear half a coont of bettered in the part he was pllitive from his chair and paced up and down the room which he had apparently adjusted the man who wrote the toom and cootedled. the dourse of the street. 
it is a
customary contraction like out of the lont lenter. it was ne it a slall sllenni pf herting in the conner of the street. 
"indied! my must be recovered."

"we have tried and a galf oo the steps; she wasched us and down in front of the forrer. i saw his motere men
and the lady; but just as he reached
her hands up his chair and paced up and down the room which he had apparently adjusted the man who wrote the toom and cootedled. the dourse of the street. 
it is a
customary contraction like out of the lont lenter. it was ne it a slall sllenni pf herting in the conner of the street. 

Text looks decent, not significantly better or worse than the previous experiment with window size = 100. 

Experiment with 3 layer model and window size = 50 is here. 

In [139]:
n_epochs = 50
batch_size = 128 
model50 = book2Model()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(device)
model50.to(device)

optimizer = optim.Adam(model50.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle = True, batch_size=batch_size)

best_model = None
best_loss = np.inf

for epoch in range(n_epochs):
    model50.train()
    for X_batch, y_batch in loader: 
        y_pred = model50(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #Validation Time
    model50.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model50(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model50.state_dict()
        print("Epoch %d: Cross-entropy: %.3f" % (epoch, loss))
torch.save([best_model, char_to_int], "single-char-50-2.pth")

Epoch 0: Cross-entropy: 150352.578
Epoch 1: Cross-entropy: 132503.484
Epoch 2: Cross-entropy: 125117.719
Epoch 3: Cross-entropy: 118761.719
Epoch 4: Cross-entropy: 116234.234
Epoch 5: Cross-entropy: 108954.469
Epoch 6: Cross-entropy: 109986.922
Epoch 7: Cross-entropy: 103127.672
Epoch 8: Cross-entropy: 99225.438
Epoch 9: Cross-entropy: 94391.773
Epoch 10: Cross-entropy: 91105.312
Epoch 11: Cross-entropy: 89058.500
Epoch 12: Cross-entropy: 85809.352
Epoch 13: Cross-entropy: 83689.523
Epoch 14: Cross-entropy: 79318.953
Epoch 15: Cross-entropy: 76596.773
Epoch 16: Cross-entropy: 74223.164
Epoch 17: Cross-entropy: 72065.070
Epoch 18: Cross-entropy: 70673.703
Epoch 19: Cross-entropy: 66093.320
Epoch 20: Cross-entropy: 70347.875
Epoch 21: Cross-entropy: 62649.438
Epoch 22: Cross-entropy: 60786.703
Epoch 23: Cross-entropy: 58023.207
Epoch 24: Cross-entropy: 56147.645
Epoch 25: Cross-entropy: 54004.727
Epoch 26: Cross-entropy: 54471.883
Epoch 27: Cross-entropy: 49944.621
Epoch 28: Cross-entrop

In [140]:
best_model, char_to_int, torch.load("single-char-50-2.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model50.load_state_dict(best_model)

<All keys matched successfully>

In [141]:
#generate a prompt here 
file50_2 = "/sherlock.txt"
raw_2txt_2 = open(file50_2, 'r', encoding = 'utf-8').read()
raw_2txt_2 = raw_2txt_2.lower()
raw_2txt_2 = raw_2txt_2[:50000]
seq_len = 50
start = np.random.randint(0, len(raw_2txt_2)-seq_len)
prompt = raw_2txt_2[start:start+seq_len]
pattern = [char_to_int[c] for c in prompt]

In [142]:
model50.eval()
print("Prompt:")
print(prompt)
print("Prompt ends here.")
print("\n")
print("Result:")
with torch.no_grad():
  for i in range(1000):
    #format input array of int into pytorch tensor 
    x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
    x = torch.tensor(x, dtype=torch.float32)
    #genreate logits as output from the model 
    pred = model(x.to(device))
    #convert logits into one character
    index = int(pred.argmax())
    result = int_to_char[index]
    print(result, end="")
    #append the new character into the prompt for the next iteration
    pattern.append(index)
    pattern = pattern[1:]

print()
print("Done.")

Prompt:
e speaks of irene adler, or when he refers to her

Prompt ends here.


Result:
cound be me that the would be the matt post."

"nh, then we have the pro house."

"i was aware of that?" asked the king had doue in a dreat scandal through the street. "it is a
customary contraction like out of the lont lenter. it was ne it a slall sllenni pf herting in the conner of the street. 
"indied! my must be recovered."

"we have tried and a galf oo the steps; she wasched us and down in front of the forrer. i saw his motere men
and the lady; but just as he reached
her hands up his chair and paced up and down the room which he had apparently adjusted the man who wrote the toom and cootedled. the dourse of the street. 
it is a
customary contraction like out of the lont lenter. it was ne it a slall sllenni pf herting in the conner of the street. 
"indied! my must be recovered."

"we have tried and a galf oo the steps; she wasched us and down in front of the forrer. i saw his motere men
and the l

Again, text generation is better with 3 layer model. The sentences here are a lot clearer and seem closer to the original text. 

Last note: the cross entropy decreased more for the 3 layer model than the 2 layer model regardless of window size. For the 2 layer model, the cross entropy was relatively similar for both window sizes. Based on the cross entropy values, the 3 layer model overall has the better performance. 